<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Visual Behavior Neuropixels Exercise 2: Active vs. Passive </h1> 

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Visual stimuli refresher

As a reminder, every recording session consisted of three major visual stimulus epochs in the following order (diagrammed below):
- An active behavior session during which the mouse performed the change detection task
- Receptive field mapping and full-field flash stimuli
- 'Passive' replay of stimulus shown during active behavior, but without the lickspout so the mouse can no longer respond.
    
</div>

<div>
<img src="https://brainmapportal-live-4cc80a57cd6e400d854-f7fdcae.divio-media.net/filer_public_thumbnails/filer_public/65/58/6558f0eb-c3c5-45e6-b645-b2e432200804/active_passive_diagram.png__710x291_q90_subsampling-2.png", width="900"/>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Descriptions of each stimulus block in <code>stimulus_presentations['stimulus_block']</code>

**block 0**: Change detection task. Natural images are flashed repeatedly and the mouse is rewarded for licking when the identity of the image changes. You can find more info about this task [here](http://portal.brain-map.org/explore/circuits/visual-behavior-neuropixels?edit&language=en).

**block 1**: Brief gray screen

**block 2**: Receptive field mapping using gabor stimuli. For more details on this stimulus consult [this notebook](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_receptive_fields.html).

**block 3**: Longer gray screen

**block 4**: Full-field flashes, shown at 80% contrast. Flashes can be black (color = -1) or white (color = 1).

**block 5**: Passive replay. Frame-for-frame replay of the stimulus shown during the change detection task (block 0), but now with the lick spout retracted so the animal can no longer engage in the task.

    
For this exercise, we will focus on **block 0** and **block 5**, the **active behavior** and the **passive replay** of the same images.
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**Exercise 2.1: Plot neural activity from VISp averaged across Active vs. Passive image changes from one ephys session**

* 2.1.1: Load an ephys session
* 2.1.2: Load neural data
* 2.1.3: Filter for good quality VISp units 
* 2.1.4: Find timepoints of image changes
* 2.1.5: Align VISp activity to changes
* 2.1.6: Find indices of Active and Passive changes
* 2.1.7: Take the mean across units and Active vs. Passive trials
* 2.1.8: Plot mean VISp activity for Active vs. Passive changes
    
Hint: we did something similar during the Tutorial, but for Novel vs. Familiar image changes
    
    
Bonus: Plot baseline-subtracted Active vs. Passive change responses
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.1: Load an ephys session**

Feel free to load the same session from the tutorial. Or if you're feeling adventurous, load a different one!
    
Hint: First, initialize the cache. Next, load the ecephys_session_table, choose a session, and load it.
    
</div>

In [2]:
import platform
platstring = platform.platform()

data_dirname = 'visual-behavior-neuropixels'
use_static = False
if 'Darwin' in platstring or 'macOS' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2022/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on AWS
    data_root = "/data/"
    data_dirname = 'visual-behavior-neuropixels-data'
    use_static = True
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2022/"

In [3]:
from allensdk.brain_observatory.behavior.behavior_project_cache.\
    behavior_neuropixels_project_cache \
    import VisualBehaviorNeuropixelsProjectCache

In [4]:
# this path should point to the location of the dataset on your platform
cache_dir = os.path.join(data_root, data_dirname)

cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(
            cache_dir=cache_dir, use_static_cache=use_static)


In [5]:
session_id = 1053941483
session = cache.get_ecephys_session(
            ecephys_session_id=session_id) #get all data that has the same session_id

/Users/christinejunhuiliu/opt/anaconda3/envs/swdb2022/lib/python3.8/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.2: Load neural data**

What do we need to load from the session object in order to plot the spiking activity of units from VISp?
    
Hint: we need information about what area they are in, along with information about what times they spiked.
    
</div>

In [20]:
stimulus_presentations = session.stimulus_presentations #this is a dataFrame
units1 = session.get_units()
channels = session.get_channels()
spikes = session.spike_times


In [21]:
unit2 = units1.merge (channels, left_on = 'peak_channel_id',right_index=True)
VISp_units = unit2[unit2.structure_acronym == 'VISp']


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.3: Filter for good quality VISp units**
    
Reminder: the index of the units table is the unit_id    

</div>

In [22]:
goodVISp_units = VISp_units[(VISp_units.quality == 'good') &
                  (VISp_units.snr > 1) &
                  (VISp_units.isi_violations < 1)]
goodVISp_units.shape

(49, 34)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**2.1.4: Find timepoints of image changes**

Hint: most stimulus presentations are not changes. Make a new table that consists of only image changes.
    
</div>

In [23]:
active_stims = stimulus_presentations[stimulus_presentations['active']]
change_stims = active_stims[active_stims.is_change==True]

In [26]:
goodVISp_units.columns

Index(['PT_ratio', 'amplitude', 'amplitude_cutoff', 'cluster_id',
       'cumulative_drift', 'd_prime', 'firing_rate', 'isi_violations',
       'isolation_distance', 'l_ratio', 'local_index', 'max_drift',
       'nn_hit_rate', 'nn_miss_rate', 'peak_channel_id', 'presence_ratio',
       'quality', 'recovery_slope', 'repolarization_slope', 'silhouette_score',
       'snr', 'spread', 'velocity_above', 'velocity_below',
       'waveform_duration', 'anterior_posterior_ccf_coordinate',
       'dorsal_ventral_ccf_coordinate', 'filtering',
       'left_right_ccf_coordinate', 'probe_channel_number',
       'probe_horizontal_position', 'probe_id', 'probe_vertical_position',
       'structure_acronym'],
      dtype='object')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**2.1.5: Align VISp activity to image changes**

We will use the <code>makePSTH</code> and <code>make_neuron_time_trials_array</code> functions, so we should define them here or import them.

</div>

In [18]:
def makePSTH(spikes, startTimes, windowDur, binSize=0.001):
    '''
    Convenience function to compute a peri-stimulus-time histogram
    (see section 7.2.2 here: https://neuronaldynamics.epfl.ch/online/Ch7.S2.html)
    INPUTS:
        spikes: spike times in seconds for one unit
        startTimes: trial start times in seconds; the first spike count 
            bin will be aligned to these times
        windowDur: trial duration in seconds
        binSize: size of spike count bins in seconds
    OUTPUTS:
        Tuple of (PSTH, bins), where:
            PSTH gives the trial-averaged spike rate for 
                each time bin aligned to the start times;
            bins are the bin edges as defined by numpy histogram
    '''
    bins = np.arange(0,windowDur+binSize,binSize)
    counts = np.zeros(bins.size-1)
    for start in startTimes:
        startInd = np.searchsorted(spikes, start)
        endInd = np.searchsorted(spikes, start+windowDur)
        counts = counts + np.histogram(spikes[startInd:endInd]-start, bins)[0]
    
    counts = counts/len(startTimes)
    return counts/binSize, bins[:-1]

In [19]:
def make_neuron_time_trials_array(units, spike_times, stim_table, 
                                   time_before, trial_duration,
                                   bin_size=0.001): #bin_size is 1 msec
    '''
    Function to make a 3D array with dimensions [neurons, time bins, trials] to store
    the spike counts for stimulus presentation trials. 
    INPUTS:
        units: dataframe with unit info (same form as session.units table)
        stim_table: dataframe whose indices are trial ids and containing a
            'start_time' column indicating when each trial began
        time_before: seconds to take before each start_time in the stim_table
        trial_duration: total time in seconds to take for each trial
        bin_size: bin_size in seconds used to bin spike counts 
    OUTPUTS:
        unit_array: 3D array storing spike counts. The value in [i,j,k] 
            is the spike count for neuron i at time bin j in the kth trial.
        time_vector: vector storing the trial timestamps for the time bins
    '''
    # Get dimensions of output array
    neuron_number = len(units)
    trial_number = len(stim_table)
    num_time_bins = int(trial_duration/bin_size)
    
    # Initialize array
    unit_array = np.zeros((neuron_number, num_time_bins, trial_number))
    
    # Loop through units and trials and store spike counts for every time bin
    for u_counter, (iu, unit) in enumerate(units.iterrows()):
        
        # grab spike times for this unit
        unit_spike_times = spike_times[iu]
        
        # now loop through trials and make a PSTH for this unit for every trial
        for t_counter, (it, trial) in enumerate(stim_table.iterrows()):
            trial_start = trial.start_time - time_before
            unit_array[u_counter, :, t_counter] = makePSTH(unit_spike_times, 
                                                            [trial_start], 
                                                            trial_duration, 
                                                            binSize=bin_size)[0]
    
    # Make the time vector that will label the time axis
    time_vector = np.arange(num_time_bins)*bin_size - time_before
    
    return unit_array, time_vector

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.6: Find indices of Active and Passive changes**

Hint: remember to use your new table of image changes, otherwise you might find the wrong indices
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.7: Take the mean across units and Active vs. Passive trials**
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.1.8: Plot mean VISp activity for Active vs. Passive changes**
    
</div>

* You should see a difference between VISp responses to Active vs. Passive changes
* The largest difference is an overall shift in firing rate. What if we want to know whether the evoked firing rate relative to the baseline is the same or different in Active vs. Passive conditions?

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Bonus: baseline-subtracted PSTHs**

* If what we are interested in is the stimulus-evoked changes in firing rate, we need to correct for the baseline.
    
* Now, before averaging over neurons, first subtract each neuron's baseline firing rate (0.2 - 0 sec before the change).

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise 2.2: Plot running and pupil area averaged across Active vs. Passive image changes from the same session**

The lick spout is retracted, so mice are no longer performing behavior and there are no licks. Let's look at other measures of behavior, running and pupil size.
    
* 2.2.1: Get running speed and eye tracking tables
* 2.2.2: Align running and pupil data to Active vs. Passive changes
* 2.2.3: Average across trials
* 2.2.4: Plot these averages
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
**2.2.1: Get runnning speed and eye tracking tables**
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**2.2.2: Align running and pupil data to Active changes**
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**2.2.3: Align running and pupil data to Passive changes**
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**2.2.4: Average across trials and plot**
    
</div>